In [12]:
# Lets start by importing all the modules and declaring the global variables
import pandas as pd
import numpy as np
import folium # map rendering library
import requests
# import k-means from clustering stage
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
pd.set_option('display.max_colwidth', -1)
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries imported.')

Libraries imported.


In [13]:
import pandas as pd
!wget -q -O 'Geospatial_Coordinates' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [8]:
df = pd.read_csv('Geospatial_Coordinates')
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df.columns = ['PostalCode', 'Latitude', 'Longitude']
df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
pd.set_option('display.max_colwidth', -1)
wiki_df = pd.read_html(link,header=0)[0]
wiki_df.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [19]:
# Renaming the columns
wiki_df.columns = ['PostalCode', 'Borough', 'Neighborhood']
# Extracting Table that has an assigned borrow
wiki_df_a = wiki_df[wiki_df['Borough'] != 'Not assigned']
# Extracting Table that has not assigned borrow
wiki_df_na = wiki_df[wiki_df['Borough'] == 'Not assigned']
# Group by postcode
wiki_df_grouped = wiki_df_a.groupby('PostalCode')
#  These are the keys by which the table is grouped
wiki_df_grouped.groups.keys()
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood']

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
def add_comma_neighbor(grp):
    '''
    This function return the borough as well as Neighbor hood for each group combined or joined with comma.
    '''
    return wiki_df_grouped.get_group(grp)['Borough'].iloc[0], ", ".join(wiki_df_grouped.get_group(grp)['Neighborhood'])
# Looping for each group and appending the result to our empty neighborhoods dataframe just created
for grp in wiki_df_grouped.groups.keys():
    borough, neighbor = add_comma_neighbor(grp)
    neighborhoods = neighborhoods.append({'PostalCode' : grp,
                                          'Borough': borough,
                                          'Neighborhood': neighbor
                                         }, ignore_index=True)
    neighborhoods.loc[neighborhoods['Neighborhood'] == 'Not assigned', 'Neighborhood'] = neighborhoods[neighborhoods['Neighborhood'] == 'Not assigned']['Borough']
   

In [20]:
 neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [21]:
neigh_latlong = neighborhoods.merge(df, how='inner', left_on=['PostalCode'], right_on=['PostalCode'])
neigh_latlong.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [22]:
neigh_latlong[neigh_latlong['PostalCode'] == 'M9L']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
96,M9L,North York,Humber Summit,43.756303,-79.565963
